<a href="https://colab.research.google.com/github/JSJeong-me/KOSA-GPT_Python-2023/blob/main/Langchain_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangChain이란?
* Language : Large Language Model의 Language
* Chain: 모듈들을 체인으로 연결하여 다른 툴의 출력을 다음 툴의 입력으로 연결하는 방식
  * 예: 검색 -> OpenAI(LLM) -> 계산
* LLM(Large Language Model)을 이용해서 뭔가 만들때 쓸수 있는 툴/라이브러리 모음.
* 파이썬과 자바스크립트 라이브러리를 제공.
* 검색, 메모리, 계산, 요약, 색인, Agent(에이전트) 등의 기능을 제공
* 모듈화로 쉽게 부품을 갈아끼울수 있음
* LLM은 기본 OpenAI (text-davinci-003) 이용

---
참고 자료:
* 파이썬 문서: https://python.langchain.com/en/latest/index.html
* 개념 문서: https://docs.langchain.com/docs/
* 소스: https://github.com/hwchase17/langchain

---

In [ ]:
#@title 기본 패키지(openai, langchain) 설치
!pip install openai
!pip install langchain

In [ ]:
#@title 기타 패키지 설치 (구글검색, 위키피디아, VectorStore, HuggingFace Embedding)
!pip install google-search-results
!pip install wikipedia
!pip install faiss-cpu # 오픈소스 벡터DB (Facebook, MIT license)
!pip install sentence_transformers # HuggingFace Embedding 사용 위해서 필요
!pip install tiktoken # Summarization 할때 필요

In [ ]:
#@title 0. API 키 설정
import os
#@markdown https://platform.openai.com/account/api-keys
OPENAI_API_KEY = "sk-" #@param {type:"string"}

#@markdown https://huggingface.co/settings/tokens
#@markdown HuggingFace에서 모델 다운로드나 클라우드 모델 사용하기 위해서 필요 (무료)
HUGGINGFACEHUB_API_TOKEN = "hf_" #@param {type:"string"}

#@markdown https://serpapi.com/manage-api-key
#@markdown 구글 검색하기 위해서 필요 (월 100회 무료)
SERPAPI_API_KEY = "6c" #@param {type:"string"}

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN
os.environ["SERPAPI_API_KEY"] = SERPAPI_API_KEY


In [ ]:
#@title 1. OpenAI LLM (text-davinci-003)

from langchain.llms import OpenAI

In [ ]:
#@title 1. OpenAI LLM (text-davinci-003)

from langchain.llms import OpenAI

llm = OpenAI(model_name='text-davinci-003', temperature=0.9)



In [ ]:
type(llm)

langchain.llms.openai.OpenAI

In [ ]:
llm('미국의 빌보드차트에 오른 한국 가수 이름을 알려 주세요.')

'\n\n- Psy의 "Gangnam Style"\n- BTS의 "DNA"\n- Blackpink의 "Ddu-Du Ddu-Du"\n- CL의 "Hello Bitches"\n- TWICE의 "Likey"\n- Big Bang의 "Fantastic Baby"'

In [ ]:
llm('2020 올림픽은 어디에서 개최 되었나요?')

'\n\n2020 동계 올림픽은 러시아의 사르도바에서 개최되었습니다.'

In [ ]:
#@title 2. ChatOpenAI LLM (gpt-3.5-turbo)

from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [ ]:
chat = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.9)
sys = SystemMessage(content="당신은 스포츠 전문 AI입니다.")
msg = HumanMessage(content='2020 하계 올림픽에서 대한민국 금메달 획득 갯수는?')

In [ ]:
aimsg = chat([sys, msg])
aimsg.content

'제가 확인해본 바에 따르면, 2020하계 올림픽은 코로나19로 인해 연기되었습니다. 새로운 일정이 결정되면 다시 확인해보도록 하겠습니다.'

In [ ]:
#@title 3. Prompt Template & chain

from langchain.prompts import PromptTemplate

In [ ]:

prompt = PromptTemplate(
    input_variables=["상품"],
    template="{상품} 만드는 회사 이름 추천해줘. 기억에 남는 한글 이름으로",
)



In [ ]:
prompt.format(상품="신발")

'신발 만드는 회사 이름 추천해줘. 기억에 남는 한글 이름으로'

In [ ]:
from langchain.chains import LLMChain
chain = LLMChain(llm=chat, prompt=prompt)

# chain.run("AI 여행 추천 서비스")
chain.run(상품="콜라")

'1. 콜라박사\n2. 콜라메이커\n3. 맥스콜라\n4. 하이콜라\n5. 콜라마스터\n6. 반짝콜라\n7. 콜라코\n8. 레드콜라\n9. 블루콜라\n10. 콜라킹\n11. 원더콜라\n12. 향긋콜라\n13. 콜라클럽\n14. 초코콜라\n15. 미스터콜라'

In [ ]:
#@title 4. ChatPromptTemplate & chain

from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

In [ ]:
chat = ChatOpenAI(temperature=0)

template="You are a helpful assisstant that tranlates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [ ]:
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])


chatchain = LLMChain(llm=chat, prompt=chat_prompt)
chatchain.run(input_language="English", output_language="Korean", text="Thanks god TGIF!")


'감사합니다. 오늘은 TGIF입니다!'

In [ ]:
#@title 5. Agents and Tools

from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

* Tool: 특정 일을 하는 역할. 예) 구글 검색, 디비 조회, Python 실행/계산.
* Agents: 어떤 툴을 어떤 순서로 실행할지 결정하는 역할 (LLM 이용)

In [ ]:
tools = load_tools(["serpapi", "llm-math"], llm=chat)
# tools = load_tools(["wikipedia", "llm-math"], llm=chat)

agent = initialize_agent(tools, llm=chat, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [ ]:
agent.run("한국의 'New Jeans' 그룹 멤버의 인원 수는? 오늘 2023년 6월 23일 현재 전체 멤버들의 나이의 합은?") # 한국의 'New Jeans' 그룹 멤버의 인원 수는? 오늘 2023년 5월 30일 현재 전체 멤버들의 나이의 합은 얼마인가요

In [ ]:
len(agent.tools)

2

In [ ]:
print(agent.tools[0].description)
print(agent.tools[1].description)

A search engine. Useful for when you need to answer questions about current events. Input should be a search query.
Useful for when you need to answer questions about math.


In [ ]:
#@title 6. Memory
from langchain import ConversationChain

conversation = ConversationChain(llm=chat, verbose=True)
conversation.predict(input="인공지능에서 Transformer가 뭐야?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 인공지능에서 Transformer가 뭐야?
AI:

> Finished chain.


'Transformer는 자연어 처리 분야에서 매우 유용한 딥러닝 모델 중 하나입니다. 이 모델은 기계 번역, 질의응답, 요약 등 다양한 자연어 처리 작업에서 사용됩니다. Transformer는 기존의 RNN, LSTM 등의 모델보다 더욱 빠르고 정확한 결과를 제공합니다. 이 모델은 구글에서 개발되었으며, 현재까지도 많은 연구자들이 이를 활용하여 다양한 자연어 처리 연구를 수행하고 있습니다.'

In [ ]:
conversation.predict(input="RNN하고 차이 설명해줘.")

In [ ]:
conversation.predict(input="Seq2Seq 와도 차이점을 비교해줘.")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 인공지능에서 Transformer가 뭐야?
AI: Transformer는 자연어 처리 분야에서 매우 유용한 딥러닝 모델 중 하나입니다. 이 모델은 기계 번역, 질의응답, 요약 등 다양한 자연어 처리 작업에서 사용됩니다. Transformer는 기존의 RNN, LSTM 등의 모델보다 더욱 빠르고 정확한 결과를 제공합니다. 이 모델은 구글에서 개발되었으며, 현재까지도 많은 연구자들이 이를 활용하여 다양한 자연어 처리 연구를 수행하고 있습니다.
Human: RNN하고 차이 설명해줘.
AI: RNN은 Recurrent Neural Network의 약자로, 시퀀스 데이터를 처리하는 데 특화된 딥러닝 모델입니다. RNN은 이전 시점의 출력값을 현재 시점의 입력값으로 사용하여 시퀀스 데이터를 처리합니다. 하지만 RNN은 시퀀스가 길어질수록 기울기 소실 문제가 발생하여 성능이 저하됩니다. 반면 Transformer는 Self-Attention 메커니즘을 사용하여 입력 시퀀스의 모든 위치를 동시에 고려할 수 있으며, 이를 통해 기울기 소실 문제를 해결합니다. 또한 Transformer는 병렬 처리가 가능하여 RNN보다 더욱 빠른 속도로 처리할 수 있습니다.
Human: Seq2Seq 와도 차이점을 비교해줘.
AI:

> Finished chain.


'Seq2Seq는 Sequence-to-Sequence의 약자로, 입력 시퀀스를 다른 시퀀스로 변환하는 딥러닝 모델입니다. 이 모델은 기계 번역, 챗봇, 요약 등 다양한 자연어 처리 작업에서 사용됩니다. Transformer는 Seq2Seq 모델의 일종으로, 입력 시퀀스와 출력 시퀀스를 모두 Self-Attention 메커니즘을 사용하여 처리합니다. 이를 통해 Seq2Seq 모델에서 발생하는 문제들을 해결할 수 있습니다. 또한 Transformer는 Seq2Seq 모델보다 더욱 빠르고 정확한 결과를 제공합니다.'

In [ ]:
conversation.memory

* ConversationBufferMemory : 대화 기록(기본)
* ConversationBufferWindowMemory : 마지막 n개의 대화만 기억
* Entity Memory : 개체에 대한 정보를 저장
* Conversation Knowledge Graph Memory: 개체의 triple 저장: (sam, 좋아하는 색, 파랑)
* ConversationSummaryMemory : 대화의 요약본을 저장
* ConversationSummaryBufferMemory : 대화 요약본 + 마지막 n토큰 기억
* ConversationTokenBufferMemory : 마지막 n토큰 기억
* VectorStore-Backed Memory : 벡터DB에 정보 저장

In [ ]:
#@title 7. Document Loaders
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader(web_path="https://ko.wikipedia.org/wiki/NewJeans")

documents = loader.load()

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)
len(docs)

# 4096 token = 3000 English word

12

In [ ]:
print(docs[-1].page_content)

원본 주소 "https://ko.wikipedia.org/w/index.php?title=NewJeans&oldid=35081890"
분류: K-pop 음악 그룹한국어 음악 그룹대한민국의 팝 음악 그룹대한민국의 댄스 음악 그룹대한민국의 걸 그룹5인조 음악 그룹2022년 결성된 음악 그룹다국적 음악 그룹엠넷 엠카운트다운 1위 수상자KBS 뮤직뱅크 1위 수상자MBC 쇼! 음악중심 1위 수상자SBS 인기가요 1위 수상자숨은 분류: 위키데이터 속성 P18을 사용하는 문서위키데이터 속성 P154를 사용하는 문서위키데이터 속성 P373을 사용하는 문서위키데이터 속성 P213을 사용하는 문서위키데이터 속성 P434를 사용하는 문서위키데이터 속성 P5034를 사용하는 문서번역 확장 필요 문서베트남어 표기를 포함한 문서위키데이터에 없는 Twitter usernameISNI 식별자를 포함한 위키백과 문서NLK 식별자를 포함한 위키백과 문서뮤직브레인즈 식별자를 포함한 위키백과 문서


 이 문서는 2023년 6월 19일 (월) 11:21에 마지막으로 편집되었습니다.
모든 문서는 크리에이티브 커먼즈 저작자표시-동일조건변경허락 4.0에 따라 사용할 수 있으며, 추가적인 조건이 적용될 수 있습니다. 자세한 내용은 이용 약관을 참고하십시오.Wikipedia®는 미국 및 다른 국가에 등록되어 있는 Wikimedia Foundation, Inc. 소유의 등록 상표입니다.


개인정보처리방침
위키백과 소개
면책 조항
행동 강령
모바일 보기
개발자
통계
쿠키 정책

내용 폭 제한 전환


---
* Web Page
* PDF
* Email
* Twitter
* DataFrame (pandas)
* YouTube
* Notion
* Google Drive
* Powerpoint, Word,
* ...
---

In [ ]:
#@title 8. Summarization
from langchain.chains.summarize import load_summarize_chain
chain = load_summarize_chain(chat, chain_type="map_reduce", verbose=True)
chain.run(docs[:-1])


* "stuff": LLM 한번에 다 보냄. 길면 오류남
* "map_reduce": 나눠서 요약, 전체 요약본 다시 요약
* "refine": (요약 + 다음 문서) => 요약
* "map_rerank": 점수매겨서 중요한거로 요약

In [ ]:
#@title 9. Embeddings and VectorStore
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings import OpenAIEmbeddings

# embeddings = OpenAIEmbeddings()
embeddings = HuggingFaceEmbeddings()

from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import FAISS

# from langchain.text_splitter import RecursiveCharacterTextSplitter
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

index = VectorstoreIndexCreator(
    vectorstore_cls=FAISS,
    embedding=embeddings,
    # text_splitter=text_splitter,
    ).from_loaders([loader])

# 파일로 저장
index.vectorstore.save_local("faiss-nj")

* Embedding : LLM에 input(text)를 넣었을때 나오는 특징 벡터 - Neural Network의 특정 메모리를 읽어서 만들고, 보통 512개 이상의 float로 만들어짐.
* 의미상 비슷한 input이 들어갔을때, 나오는 특징 벡터가 유사한 특징을 갖고, 거리상 가깝다.
* OpenAI에서 text를 넣으면 Embedding을 돌려주는 API를 제공한다. (다만 비용 발생)
* 로컬에서 유사한 알고리즘을 돌릴수 있다. (다만 똑같은 벡터는 아니고, openAI 임베딩과는 비교할수 없다)
* HuggingFaceEmbeddings 사용하면 로컬에서 돌릴수 있고, GPU 있다면 가속도 지원함.
* Vector Database는 Embedding 벡터와 텍스트를 저장하는 DB
  * Pinecone 서비스 (유료, 무료 제한적)
  * FAISS (페이스북, 오픈소스, 로컬)
  * Embeddings -> 텍스트를 저장
  * Embeddings로 검색할수 있으며, 거리상 가장 가까운 항목을 가져올수 있음. (거리를 구하는 방법은 여러가지)
  * 유사한 의미를 갖는 텍스트에 대한 Embedding 벡터는 거리상 가깝다.

In [ ]:
index.query("뉴진스의 데뷔곡은?", llm=chat, verbose=True)



> Entering new  chain...

> Finished chain.


'뉴진스(NewJeans)의 데뷔곡은 "Attention"입니다.'

In [ ]:
index.query("뉴진스의 데뷔 멤버는?", llm=chat, verbose=True)



> Entering new  chain...

> Finished chain.


'뉴진스(NewJeans)는 2022년 7월 22일에 데뷔한 대한민국의 5인조 걸 그룹입니다. 멤버는 민지하, 니엘, 해린, 혜인, 모지혜입니다.'

In [ ]:
index.query("멤버의 나이는?", llm=chat, verbose=True)



> Entering new  chain...

> Finished chain.


'위키백과에서는 멤버들의 나이에 대한 정보가 제공되지 않고 있습니다. 따라서, 현재는 멤버들의 나이에 대한 정보를 알 수 없습니다.'

In [ ]:
index.query("멤버의 나이는? (오늘은 2023년 5월 30일)", llm=chat, verbose=True)



> Entering new  chain...

> Finished chain.


'민지는 2004년 5월 7일생으로 19살, 하니는 2004년 10월 6일생으로 18살, 다니엘은 2005년 4월 11일생으로 18살, 해린은 2006년 5월 15일생으로 17살, 혜인은 2008년 4월 21일생으로 15살입니다.'

In [ ]:
#@title FAISS 벡터DB 디스크에서 불러오기
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

fdb = FAISS.load_local("faiss-nj", embeddings)
index2 = VectorStoreIndexWrapper(vectorstore=fdb)

In [ ]:
index2.query("뉴진스의 데뷔 멤버는?", llm=chat, verbose=True)



> Entering new  chain...

> Finished chain.


'뉴진스(NewJeans)는 2022년 7월 22일에 데뷔한 대한민국의 5인조 걸 그룹입니다. 멤버는 민지하, 니엘, 해린, 혜인, 모지혜입니다.'